# Radioactive Decay Energy

Within the ejecta of a supernova, the gammarays largely come from the decay of $^{56}Ni$ into $^{56}Co$.
This releases a large amount of energy. To understand how much energy is produced from this decay we look at the amount of energy 1 g of $^{56}Ni$ produces in 10 days

The equation for radioactive decay is 
$$N(t) = N_\mathrm{0} \exp\left(-\lambda t \right)$$
$N(t)$ is the number of atoms after time t, $N_\mathrm{0}$ is the initial number of atoms, and $\lambda$ is the half life.

Using the molar mass, we find that 1g of $^{56}Ni$ has $1.0767e22$ atoms. The half life of $^{56}Ni$ is 6.10 days. Plugging these numbers into the equation we can find the number of atoms that have decayed after 10 days.

In [8]:
import numpy as np
from astropy import units as u
from astropy import constants as const

#initial number of atoms
n_0 = 1.0767e22
#half life
t_half = 6.10
#time in days
time = 10

#number of atoms keft after 10 days
n_final = n_0 * np.exp(-t_half * time)

#number of atoms that have decayed
n_decays = n_0 - n_final

The gammarays that are released during this radiocative decay can be released from a number of energy level transitions. Each transition has an associated energy and a probability out of 100 decays. From the combination of all of these tranisitions and their probabilities, the total energy released per decay is $1.75 MeV$ (Nadyozhin 1994).

In [ ]:
#energy per decay in eV
energy_per_decay = 1.75e6

total_energy = n_decays * energy_per_decay

print("The total energy released from radioactive decay of 1g of 56Ni after 10 days is:", total_energy, "eV")

The total energy released from radioactive decay of 1g of 56Ni after 10 days is: 1.884225e+28 eV
